# Session 7 : Language Model (N Grams)

In [1]:
import pandas as pd 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

class NGramLanguageModel : 
    def __init__(self,n):
        self.n = n
        self.vectorizer = CountVectorizer(analyzer='word',ngram_range=(n,n))
        
    def fit_transform(self,corpus):
        return self.vectorizer.fit_transform(corpus)
    
    def transform(self,corpus):
        return self.vectorizer.transform(corpus)

def calculate_cosine_similarity_matrix(matrix, query_v):
    similarities = cosine_similarity(matrix, query_v)
    return similarities

corpus = [
    "This is a sample document.",
    "Here is another document.",
    "And this is a third document."
]

query = "This is the query text"

n = 1
document_index = 0

ngram_model = NGramLanguageModel(n)

matrix = ngram_model.fit_transform(corpus)
query_v = ngram_model.transform([query])

print(f"{n}-Gram Model:")
data = matrix.A
 
print(pd.DataFrame(matrix.A, columns=ngram_model.vectorizer.get_feature_names_out()))
print(query_v.A)

1-Gram Model:
   and  another  document  here  is  sample  third  this
0    0        0         1     0   1       1      0     1
1    0        1         1     1   1       0      0     0
2    1        0         1     0   1       0      1     1
[[0 0 0 0 1 0 0 1]]


In [2]:
similarities = calculate_cosine_similarity_matrix(matrix, query_v)

data = {'Document': corpus, 'Similarity': similarities.flatten()}
df = pd.DataFrame(data)

print(query)
df

This is the query text


,Document,Similarity
0,This is a sample document.,0.707107
1,Here is another document.,0.353553
2,And this is a third document.,0.632456


# Session 8 : Word Embeddings

In [3]:
import pandas as pd
import sklearn as sk
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jonathanmaverick/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/jonathanmaverick/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
first_sentence = "It is going to rain today"
second_sentence = "Today I am not going outside"

first_sentence = first_sentence.split(" ")
second_sentence = second_sentence.split(" ")
total = set(first_sentence).union(set(second_sentence))

print(total)

{'is', 'today', 'to', 'I', 'am', 'It', 'going', 'outside', 'Today', 'not', 'rain'}


In [5]:
wordDictA = dict.fromkeys(total, 0)
wordDictB = dict.fromkeys(total, 0)

for word in first_sentence:
    wordDictA[word]+=1

for word in second_sentence:
    wordDictB[word]+=1

In [6]:
pd.DataFrame([wordDictA, wordDictB])

,is,today,to,I,am,It,going,outside,Today,not,rain
0,1,1,1,0,0,1,1,0,0,0,1
1,0,0,0,1,1,0,1,1,1,1,0


In [7]:
def computeTF(wordDict, doc):
    tfDict = {}
    corpusCount = len(doc)
    for word, count in wordDict.items():
        tfDict[word] = count/float(corpusCount)
    return (tfDict)

tfFirst = computeTF(wordDictA, first_sentence)
tfSecond = computeTF(wordDictB, second_sentence)

tf = pd.DataFrame([tfFirst, tfSecond])
tf

,is,today,to,I,am,It,going,outside,Today,not,rain
0,0.166667,0.166667,0.166667,0.000000,0.000000,0.166667,0.166667,0.000000,0.000000,0.000000,0.166667
1,0.000000,0.000000,0.000000,0.166667,0.166667,0.000000,0.166667,0.166667,0.166667,0.166667,0.000000


In [8]:
filtered_sentence = []
for word in wordDictA:
    if str(word) not in set (stopwords.words('english')):
        filtered_sentence.append(word)
        
filtered_sentence

['today', 'I', 'It', 'going', 'outside', 'Today', 'rain']

In [9]:
def computeIDF(docList):
    import math
    idfDict = {}
    N = len(docList)
    
    idfDict = dict.fromkeys(docList[0].keys(), 0)
    for word, val in idfDict.items():
        idfDict[word] = math.log10(N / (float(val) + 1))
    
    idf_df = pd.DataFrame(list(idfDict.items()), columns=['Word', 'IDF'])
    
    return idf_df

idfs = computeIDF([wordDictA, wordDictB])
idfs

,Word,IDF
0,is,0.30103
1,today,0.30103
2,to,0.30103
3,I,0.30103
4,am,0.30103
5,It,0.30103
6,going,0.30103
7,outside,0.30103
8,Today,0.30103
9,not,0.30103


In [10]:
def computeTFIDF(tfbow, idfs):
    tfidf = {}
    for word, val in tfbow.items():
        tfidf[word] = val*idfs.loc[idfs['Word'] == word, 'IDF'].values[0]
    return tfidf

idfFirst = computeTFIDF(tfFirst, idfs)
idfSecond = computeTFIDF(tfSecond, idfs)

idf = pd.DataFrame([idfFirst, idfSecond])
idf

,is,today,to,I,am,It,going,outside,Today,not,rain
0,0.050172,0.050172,0.050172,0.000000,0.000000,0.050172,0.050172,0.000000,0.000000,0.000000,0.050172
1,0.000000,0.000000,0.000000,0.050172,0.050172,0.000000,0.050172,0.050172,0.050172,0.050172,0.000000


In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np 
from numpy.linalg import norm

In [12]:
Document1 = "It is going to rain today"
Document2 = "Today I am not going outside"

Doc = [Document1, Document2]
print(Doc)

['It is going to rain today', 'Today I am not going outside']


# Session 9 : Grammar Parsing with NLTK

In [15]:
import nltk
from nltk import CFG
from nltk.parse import ChartParser

def demonstrate_nlp_parsing(sentence, grammar):
    words = nltk.word_tokenize(sentence)
    parser = ChartParser(grammar)
    
    try:
        parses = list(parser.parse(words))
        if parses:
            for tree in parser.parse(sentence.split()):
                print(tree, "\n")
                tree.pretty_print()
        else :
            print("No parse tree found")
    except nltk.parse.api.ParserError as e:
        print(f"Error during parsing {e}")
        
nlp_grammar = CFG.fromstring("""
    S -> NP VP                   
    NP -> Det N | Det N PP | 'I'
    VP -> V NP | VP PP
    PP -> P NP
    Det -> 'the' | 'a'
    N -> 'cat' | 'dog' | 'park' | 'cookie'
    V -> 'chased' | 'saw' | 'ate' 
    P -> 'in' | 'on' | 'with'
""")

nlp_sentences = [
    "the cat chased the dog",
    "i saw a cookie",
    "the dog ate a cookie in the park",
    "i chased the cat with the cookie"
]

for i, sentence in enumerate(nlp_sentences):
    print(f"Sentence {i+1}: {nlp_sentences[i]}")
    demonstrate_nlp_parsing(sentence, nlp_grammar)
    print("\n")

Sentence 1: the cat chased the dog
(S (NP (Det the) (N cat)) (VP (V chased) (NP (Det the) (N dog)))) 

              S               
      ________|_____           
     |              VP        
     |         _____|___       
     NP       |         NP    
  ___|___     |      ___|___   
Det      N    V    Det      N 
 |       |    |     |       |  
the     cat chased the     dog



Sentence 2: i saw a cookie


AttributeError: module 'nltk.parse.api' has no attribute 'ParserError'